# YOLO Object Detection

## Installation

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
!chmod +x ./darknet
!ls darknet

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
with open('darknet/Makefile', 'r') as file_:
  file_content = file_.read()
file_content = file_content.replace('GPU=0', 'GPU=1').replace('CUDNN=0', 'CUDNN=1').replace('OPENCV=0', 'OPENCV=1')

f = open('darknet/Makefile', 'w')
f.write(file_content)
f.close()

In [ ]:
%cd darknet
!make

## Training

In [ ]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

In [ ]:
# only for google colab
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<key>"

In [ ]:
!kaggle datasets download -d tannergi/microcontroller-detection --unzip

In [ ]:
!mv 'Microcontroller Detection' microcontroller-detection

In [ ]:
%cd 'microcontroller-detection'
!wget https://raw.githubusercontent.com/TannerGilbert/YOLO-Tutorials/master/YOLO-Object-Detection-with-Darknet/convert_voc_to_yolo.py

In [ ]:
!python convert_voc_to_yolo.py

In [ ]:
classes_file = open('custom.names', 'w')
classes = ['Arduino_Nano', 'Heltec_ESP32_Lora', 'ESP8266', 'Raspberry_Pi_3']
num_classes = len(classes)
for c in classes:
  classes_file.write(c + '\n')
classes_file.close()

In [ ]:
%cd /content/darknet

In [ ]:
config_file = open('detector.data', 'w')

config_file.write(f'classes={num_classes}\n')
config_file.write('train=/content/darknet/microcontroller-detection/train.txt\n')
config_file.write('valid=/content/darknet/microcontroller-detection/test.txt\n')
config_file.write('names=/content/darknet/microcontroller-detection/custom.names\n')
config_file.write('backup=backup/')

config_file.close()

In [ ]:
!cat detector.data

In [ ]:
!cp cfg/yolov3.cfg yolov3-custom.cfg
with open('yolov3-custom.cfg', 'r') as file_:
  file_content = file_.read()
file_content = file_content.replace('batch=1', 'batch=64').replace('subdivisions=1', 'subdivisions=16').replace('max_batches = 500200', 'max_batches = 4000').replace('steps=400000,450000', 'steps=3200,3600').replace('classes=80', f'classes={num_classes}').replace('filters=255', f'filters={(num_classes + 5) * 3}')

f = open('yolov3-custom.cfg', 'w')
f.write(file_content)
f.close()

In [ ]:
!cat yolov3-custom.cfg

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!./darknet detector train /content/darknet/detector.data /content/darknet/yolov3-custom.cfg /content/darknet/darknet53.conv.74 -dont_show

In [ ]:
!ls -lah /content/darknet/backup/yolov3-custom_last.weights

## Convert YOLO model to Tensorflow frozen model

In [ ]:
#clone a repo that helps with the conversion
!git clone https://github.com/mystic123/tensorflow-yolo-v3.git

In [ ]:
%cd tensorflow-yolo-v3/
!git checkout ed60b90

In [ ]:
!python3 convert_weights_pb.py --class_names /content/darknet/data/coco.names --data_format NHWC --weights_file /content/darknet/backup/yolov3-custom_last.weights --class_names /content/darknet/microcontroller-detection/custom.names

## Install OpenVINO 2021.2

In [ ]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## downnload installation files
!wget https://registrationcenter-download.intel.com/akdlm/irc_nas/17504/l_openvino_toolkit_p_2021.2.185.tgz
path = "l_openvino_toolkit_p_2021.2.185.tgz"
## install openvino
!tar xf "{path}"
%cd l_openvino_toolkit_p_2021.2.185/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

In [ ]:
%cd /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/install_prerequisites/
!./install_prerequisites_tf.sh

## Convert TF model to OpenVINO IR

In [ ]:
%%writefile /content/darknet/yolo_v3.json
[
  {
    "id": "TFYOLOV3",
    "match_kind": "general",
    "custom_attributes": {
      "classes": 4,
      "anchors": [10, 13, 16, 30, 33, 23, 30, 61, 62, 45, 59, 119, 116, 90, 156, 198, 373, 326],
      "coords": 4,
      "num": 9,
      "masks":[[6, 7, 8], [3, 4, 5], [0, 1, 2]],
      "entry_points": ["detector/yolo-v3/Reshape", "detector/yolo-v3/Reshape_4", "detector/yolo-v3/Reshape_8"]
    }
  }
]

In [ ]:
!cat /content/darknet/yolo_v3.json

In [ ]:
output_dir = '/content/yolov3'

!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh && \
    python /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/mo.py \
      --input_model /content/darknet/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb \
      --tensorflow_use_custom_operations_config /content/darknet/yolo_v3.json \
      --batch 1 \
      --data_type FP16 \
      --reverse_input_channel \
      --output_dir {output_dir}

In [ ]:
!ls -lah {output_dir}/frozen_darknet_yolov3_model.bin

## Compile the IR model to a .blob for use on DepthAI modules/platform

In [ ]:
import requests

url = "http://69.164.214.171:8083/compile"  # change if running against other URL

payload = {
    'compiler_params': '-ip U8 -VPU_NUMBER_OF_SHAVES 8 -VPU_NUMBER_OF_CMX_SLICES 8',
    'compile_type': 'myriad'
}
files = [
    ('definition', open(f'{output_dir}/frozen_darknet_yolov3_model.xml', 'rb')),
    ('weights', open(f'{output_dir}/frozen_darknet_yolov3_model.bin', 'rb'))
]
params = {
    'version': '2021.1',  # OpenVINO version, can be "2021.1", "2020.4", "2020.3", "2020.2", "2020.1", "2019.R3"
}

response = requests.post(url, data=payload, files=files, params=params)

with open(f'{output_dir}/model.blob', 'wb') as f:
  f.write(response.content)

In [ ]:
!ls -lah {output_dir}/model.blob

In [ ]:
from google.colab import files
files.download(f'{output_dir}/model.blob')